<a href="https://colab.research.google.com/github/lahirumanulanka/ann-visual-emotion/blob/main/notebooks/emo_CNN_Baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1) Setup & Imports

In [1]:
import os, math, json, random, time
from pathlib import Path

import numpy as np
import pandas as pd
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from torchvision import transforms, models

from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt

In [2]:
SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Device:', device)

Device: cpu


## 2) Config & Label Map

In [3]:
BASE_DIR = "/content/ann-visual-emotion/data/processed/EmoSet_splits"
BASE_DATA_URL = "/content/ann-visual-emotion"

# File paths
CSV_TRAIN = "../data/processed/EmoSet_splits/train.csv"
CSV_VAL   = "../data/processed/EmoSet_splits/val.csv"
CSV_TEST  = "../data/processed/EmoSet_splits/test.csv"
LABEL_MAP_JSON = "../data/processed/EmoSet_splits/label_map.json"
STATS_JSON     = "../data/processed/EmoSet_splits/stats.json"   

# Column names in the CSVs
COL_IMAGE = 'image'
COL_LABEL = 'label'

# Training hyperparameters
IMG_SIZE     = 224
BATCH_SIZE   = 32
EPOCHS       = 20
BASE_LR      = 3e-4
WEIGHT_DECAY = 1e-4
LABEL_SMOOTH = 0.05
USE_SAMPLER  = True            # balanced mini-batches
MIXUP_ALPHA  = 0.0             # set to 0.2~0.4 to enable MixUp

# Checkpoint path
CKPT_PATH = 'best_resnet18_balanced.pt'

## 3) Load CSVs & Quick EDA

In [4]:
train_df = pd.read_csv(CSV_TRAIN)
val_df   = pd.read_csv(CSV_VAL)
test_df  = pd.read_csv(CSV_TEST)

print("Train:", len(train_df), "Val:", len(val_df), "Test:", len(test_df))
print(train_df.head())

Train: 28821 Val: 5653 Test: 1413
                filepath  label  label_id  split
0  train/angry/14147.jpg  angry         0  train
1  train/angry/24084.jpg  angry         0  train
2    train/angry/823.jpg  angry         0  train
3  train/angry/19463.jpg  angry         0  train
4  train/angry/30797.jpg  angry         0  train


In [5]:

def _auto_column(df, candidates):
    for c in candidates:
        if c in df.columns:
            return c
    raise KeyError(f"None of {candidates} found. Columns: {list(df.columns)}")


def load_csvs(csv_train, csv_val, csv_test=None, col_image='image_path', col_label='label'):
    tr = pd.read_csv(csv_train)
    va = pd.read_csv(csv_val)
    te = pd.read_csv(csv_test) if csv_test and os.path.exists(csv_test) else None

    if col_image not in tr.columns:
        col_image = _auto_column(tr, ['image_path','image','filepath','path','img_path','img'])
    if col_label not in tr.columns:
        col_label = _auto_column(tr, ['label','emotion','target','y'])

    for df in (tr, va) + ((te,) if te is not None else ()):
        df[col_image] = df[col_image].astype(str)
        df[col_label] = df[col_label].astype(str)

    labels = sorted(tr[col_label].unique().tolist())
    label_to_idx = {l:i for i,l in enumerate(labels)}
    idx_to_label = {i:l for l,i in label_to_idx.items()}

    tr['y'] = tr[col_label].map(label_to_idx)
    va['y'] = va[col_label].map(label_to_idx)
    if te is not None:
        before = len(te)
        te = te[te[col_label].isin(labels)].copy()
        if len(te) < before:
            print(f"[WARN] Dropped {before-len(te)} test rows with unseen labels.")
        te['y'] = te[col_label].map(label_to_idx)

    return tr, va, te, label_to_idx, idx_to_label, col_image, col_label


def resolve_path(p: str) -> str:
    """Return an absolute path. If not existing, try join with BASE_DATA_URL."""
    # Already absolute and exists
    if os.path.isabs(p) and os.path.exists(p):
        return p
    # Try relative to BASE_DATA_URL
    cand = os.path.join(BASE_DATA_URL, p.lstrip('/'))
    if os.path.exists(cand):
        return cand
    # As last resort, return original (may be handled by PIL if path becomes valid in Colab mount)
    return p


class ImageCSVDataset(Dataset):
    def __init__(self, df, img_col, y_col='y', transform=None):
        self.df = df.reset_index(drop=True)
        self.img_col = img_col
        self.y_col = y_col
        self.tfm = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        path = resolve_path(row[self.img_col])
        y = int(row[self.y_col])
        with Image.open(path) as im:
            im = im.convert('RGB')
        if self.tfm:
            im = self.tfm(im)
        return im, y


def compute_class_weights(y_int, power=0.5):
    counts = np.bincount(y_int)
    counts[counts==0] = 1
    w = (1.0 / counts) ** power
    w = w / w.sum() * len(counts)
    return torch.tensor(w, dtype=torch.float32)


def make_sampler(y_int):
    class_count = np.bincount(y_int)
    class_count[class_count==0] = 1
    class_weight = 1.0 / class_count
    sample_weight = np.array([class_weight[y] for y in y_int])
    return WeightedRandomSampler(weights=torch.from_numpy(sample_weight).double(),
                                 num_samples=len(sample_weight),
                                 replacement=True)


In [6]:
# Load CSVs
train_df, val_df, test_df, label_to_idx, idx_to_label, COL_IMAGE, COL_LABEL = load_csvs(
    CSV_TRAIN, CSV_VAL, CSV_TEST, COL_IMAGE, COL_LABEL
)
NUM_CLASSES = len(label_to_idx)
print('Classes:', label_to_idx)

# Transforms (ImageNet mean/std)
IMAGENET_MEAN = [0.485, 0.456, 0.406]
IMAGENET_STD = [0.229, 0.224, 0.225]

train_tfms = transforms.Compose([
    transforms.RandomResizedCrop(IMG_SIZE, scale=(0.6, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.1, hue=0.05),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD),
])
val_tfms = transforms.Compose([
    transforms.Resize(IMG_SIZE + 32),
    transforms.CenterCrop(IMG_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD),
])

train_ds = ImageCSVDataset(train_df, img_col=COL_IMAGE, transform=train_tfms)
val_ds   = ImageCSVDataset(val_df,   img_col=COL_IMAGE, transform=val_tfms)
test_ds  = ImageCSVDataset(test_df,  img_col=COL_IMAGE, transform=val_tfms) if test_df is not None else None

# Sampler / Loaders
sampler = make_sampler(train_df['y'].values) if USE_SAMPLER else None
train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=(sampler is None),
                          sampler=sampler, num_workers=4, pin_memory=True)
val_loader   = DataLoader(val_ds,   batch_size=BATCH_SIZE, shuffle=False,
                          num_workers=4, pin_memory=True)
test_loader  = DataLoader(test_ds,  batch_size=BATCH_SIZE, shuffle=False,
                          num_workers=4, pin_memory=True) if test_ds is not None else None

# Class weights (for CE)
class_weights = compute_class_weights(train_df['y'].values, power=0.5).to(device)
print('Class weights:', class_weights.cpu().numpy().round(3))

Classes: {'angry': 0, 'disgust': 1, 'fear': 2, 'happy': 3, 'neutral': 4, 'sad': 5, 'surprise': 6}
Class weights: [0.807 2.443 0.796 0.603 0.723 0.726 0.901]


In [8]:
def build_model(num_classes):
    # Use pretrained weights if available, else None
    try:
        weights = models.ResNet18_Weights.DEFAULT
    except AttributeError:
        weights = None
    net = models.resnet18(weights=weights)
    # freeze all
    for p in net.parameters():
        p.requires_grad = False
    # unfreeze last block
    for p in net.layer4.parameters():
        p.requires_grad = True
    in_features = net.fc.in_features
    net.fc = nn.Linear(in_features, num_classes)
    return net

model = build_model(NUM_CLASSES).to(device)

# Optimizer / Scheduler
opt = torch.optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()),
                        lr=BASE_LR, weight_decay=WEIGHT_DECAY)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(opt, T_max=10)
scaler = torch.cuda.amp.GradScaler(enabled=torch.cuda.is_available())


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /Users/lahirumunasinghe/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100.0%
100.0%
/var/folders/zy/l_s5vzv53fz8ysl24mcw4m_h0000gn/T/ipykernel_25390/3165508055.py:24: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=torch.cuda.is_available())
/var/folders/zy/l_s5vzv53fz8ysl24mcw4m_h0000gn/T/ipykernel_25390/3165508055.py:24: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=torch.cuda.is_available())


In [9]:
def mixup(images, labels, alpha=0.2):
    if alpha <= 0.0:
        return images, labels, labels, 1.0, False
    lam = np.random.beta(alpha, alpha)
    index = torch.randperm(images.size(0), device=images.device)
    mixed_x = lam * images + (1 - lam) * images[index, :]
    y_a, y_b = labels, labels[index]
    return mixed_x, y_a, y_b, lam, True

In [10]:
def epoch_pass(loader, train=True):
    if train:
        model.train()
    else:
        model.eval()
    all_preds, all_tgts = [], []
    total_loss, total_n = 0.0, 0

    for ims, ys in loader:
        ims, ys = ims.to(device), ys.to(device)
        if train:
            opt.zero_grad(set_to_none=True)
        with torch.cuda.amp.autocast(enabled=torch.cuda.is_available()):
            ims2, y_a, y_b, lam, used_mix = mixup(ims, ys, alpha=MIXUP_ALPHA if train else 0.0)
            logits = model(ims2)
            if used_mix:
                loss = lam*F.cross_entropy(logits, y_a, weight=class_weights, label_smoothing=LABEL_SMOOTH) + \
                       (1-lam)*F.cross_entropy(logits, y_b, weight=class_weights, label_smoothing=LABEL_SMOOTH)
            else:
                loss = F.cross_entropy(logits, ys, weight=class_weights, label_smoothing=LABEL_SMOOTH)
        if train:
            scaler.scale(loss).backward()
            scaler.step(opt)
            scaler.update()

        total_loss += loss.item() * ims.size(0)
        total_n    += ims.size(0)
        all_preds.append(logits.detach().cpu())
        all_tgts.append(ys.detach().cpu())

    if train:
        scheduler.step()

    y_true = torch.cat(all_tgts).numpy()
    y_pred = torch.cat(all_preds).argmax(1).numpy()

    # macro-F1 (simple sklearn computation)
    from sklearn.metrics import f1_score, accuracy_score
    f1 = f1_score(y_true, y_pred, average='macro', zero_division=0)
    acc = accuracy_score(y_true, y_pred)
    return total_loss/total_n, f1, acc, y_true, y_pred

In [ ]:

best_f1 = 0.0
history = []

for epoch in range(1, EPOCHS+1):
    t0 = time.time()
    tr_loss, tr_f1, tr_acc, _, _ = epoch_pass(train_loader, train=True)
    va_loss, va_f1, va_acc, y_true, y_pred = epoch_pass(val_loader, train=False)
    dt = time.time()-t0
    print(f"Epoch {epoch:02d} | train_loss {tr_loss:.4f} F1 {tr_f1:.3f} acc {tr_acc:.3f} || val_loss {va_loss:.4f} F1 {va_f1:.3f} acc {va_acc:.3f}  [{dt:.1f}s]")
    history.append((epoch, tr_loss, tr_f1, tr_acc, va_loss, va_f1, va_acc))

    if va_f1 > best_f1:
        best_f1 = va_f1
        torch.save({'model': model.state_dict(),
                    'label_to_idx': label_to_idx,
                    'idx_to_label': idx_to_label}, CKPT_PATH)
        print('  -> saved best checkpoint')

# Optionally unfreeze more and continue training if plateau (run again with different setup)


/Users/lahirumunasinghe/Documents/DataScience/ann-visual-emotion/.venv/lib/python3.13/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
Traceback (most recent call last):
  File "<string>", line 1, in <module>
    from multiprocessing.spawn import spawn_main; spawn_main(tracker_fd=80, pipe_handle=94)
                                                  ~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/Cellar/python@3.13/3.13.2/Frameworks/Python.framework/Versions/3.13/lib/python3.13/multiprocessing/spawn.py", line 122, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/opt/homebrew/Cellar/python@3.13/3.13.2/Frameworks/Python.framework/Versions/3.13/lib/python3.13/multiprocessing/spawn.py", line 132, in _main
    self = reduction.pickle.load(from_parent)
AttributeError: Can't get attribute 'ImageCSVDataset' on <module